In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)


C:\Users\ilya2\Desktop\VStudio\Python\TheBestMan\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [2]:
df = pd.read_csv("../data/code-review-dataset-clear.csv")

In [3]:
X_train, X_val = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df["is_toxic"]
)

train_ds = Dataset.from_pandas(
    X_train[["message", "is_toxic"]].rename(columns={"message": "text", "is_toxic": "labels"}),
    preserve_index=False
)
val_ds = Dataset.from_pandas(
    X_val[["message", "is_toxic"]].rename(columns={"message": "text", "is_toxic": "labels"}),
    preserve_index=False
)

# RoBERTa


In [4]:
model_name = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
    )

train_ds = train_ds.map(tokenize_batch, batched=True)
val_ds   = val_ds.map(tokenize_batch, batched=True)

cols_to_keep = ["input_ids", "attention_mask", "labels"]
train_ds = train_ds.remove_columns([c for c in train_ds.column_names if c not in cols_to_keep])
val_ds   = val_ds.remove_columns([c for c in val_ds.column_names   if c not in cols_to_keep])


Map: 100%|██████████| 2581/2581 [00:00<00:00, 30947.77 examples/s]


In [5]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "non_toxic", 1: "toxic"},
    label2id={"non_toxic": 0, "toxic": 1},
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", pos_label=1, zero_division=0)
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}



In [ ]:
args = TrainingArguments(
    output_dir="./roberta_toxic_cls",
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,
    logging_steps=50,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [8]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.187500,0.210977,0.918249,0.803313,0.769841,0.786221
2,0.161100,0.265613,0.920186,0.834081,0.738095,0.783158
3,0.133200,0.313939,0.917861,0.793173,0.783730,0.788423


TrainOutput(global_step=1938, training_loss=0.20639460989811348, metrics={'train_runtime': 87.9994, 'train_samples_per_second': 351.889, 'train_steps_per_second': 22.023, 'total_flos': 2036874235069440.0, 'train_loss': 0.20639460989811348, 'epoch': 3.0})

In [9]:
eval_metrics = trainer.evaluate()
print(eval_metrics)

{'eval_loss': 0.31393930315971375, 'eval_accuracy': 0.9178612940720651, 'eval_precision': 0.7931726907630522, 'eval_recall': 0.7837301587301587, 'eval_f1': 0.7884231536926147, 'eval_runtime': 1.4042, 'eval_samples_per_second': 1838.001, 'eval_steps_per_second': 57.682, 'epoch': 3.0}
